<a href="https://colab.research.google.com/github/FarshadBorjalizade/ImagePprocessing-ComputerVision/blob/main/cnn_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True )

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import timeit

In [ ]:
(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()

In [ ]:
## Shape, Dtype, Normalize, One-Hot
train_data.shape

(60000, 28, 28)

In [ ]:
train_data = train_data.reshape((-1, 28, 28, 1)).astype(np.float32) / 255.
test_data = test_data.reshape((-1, 28, 28, 1)).astype(np.float32) / 255.
train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, ReLU
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

In [ ]:
# Input
i = Input((28, 28, 1), name='Input layer')

# CNN
x = Conv2D(filters=16 , kernel_size=(5, 5) , strides=1, padding='same', name='Block1_Conv1')(i)
x = ReLU()(x)
x = MaxPooling2D(pool_size=(2,2), strides=2, padding='same', name='Block1_Max_pool_1')(x)

x = Conv2D(filters=32 , kernel_size=(3, 3), strides=1 , padding='same')(x)
x = ReLU()(x)
x = MaxPooling2D(pool_size=(2,2), strides=2, padding='same')(x)

x = Conv2D(filters=64 , kernel_size=(3, 3), strides=1, padding='same')(x)
x = ReLU()(x)
x = MaxPooling2D(pool_size=(2,2), strides=2, padding='same')(x)

# Classifier

# option 1
#x = GlobalAveragePooling2D()(x)

# option 2
x = Flatten()(x)
c = Dense(units=128, activation='relu')(x)

p = Dense(units=10, activation='softmax')(c)

# Create model
cnn_model = tf.keras.models.Model(i, p)
cnn_model.summary()

start_time = timeit.default_timer()
cnn_model(np.random.rand(32, 28, 28, 1))
end_tmime = timeit.default_timer()
print('Time = {}'.format(end_tmime - start_time))

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input layer (InputLayer)    [(None, 28, 28, 1)]       0         
                                                                 
 Block1_Conv1 (Conv2D)       (None, 28, 28, 16)        416       
                                                                 
 re_lu_36 (ReLU)             (None, 28, 28, 16)        0         
                                                                 
 Block1_Max_pool_1 (MaxPooli  (None, 14, 14, 16)       0         
 ng2D)                                                           
                                                                 
 conv2d_34 (Conv2D)          (None, 14, 14, 32)        4640      
                                                                 
 re_lu_37 (ReLU)             (None, 14, 14, 32)        0         
                                                          

In [ ]:
start_time = timeit.default_timer()
cnn_model(np.random.rand(32, 28, 28, 1))
end_tmime = timeit.default_timer()
print('Time = {}'.format(end_tmime - start_time))

Time = 0.01331871099955606


In [ ]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
save_best = tf.keras.callbacks.ModelCheckpoint('best_cnn_model.h5', save_best_only=True)

In [ ]:
cnn_model.fit(x=train_data, y=train_labels, epochs=10,
             batch_size=32, validation_split=0.2, callbacks=[save_best])

Epoch 1/10
1500/1500 [==============================] - 53s 34ms/step - loss: 0.1611 - categorical_accuracy: 0.9496 - val_loss: 0.0692 - val_categorical_accuracy: 0.9795
Epoch 2/10
1500/1500 [==============================] - 49s 33ms/step - loss: 0.0512 - categorical_accuracy: 0.9842 - val_loss: 0.0696 - val_categorical_accuracy: 0.9786
Epoch 3/10
1500/1500 [==============================] - 49s 33ms/step - loss: 0.0379 - categorical_accuracy: 0.9878 - val_loss: 0.0439 - val_categorical_accuracy: 0.9866
Epoch 4/10
1500/1500 [==============================] - 49s 33ms/step - loss: 0.0289 - categorical_accuracy: 0.9910 - val_loss: 0.0345 - val_categorical_accuracy: 0.9897
Epoch 5/10
1500/1500 [==============================] - 49s 33ms/step - loss: 0.0239 - categorical_accuracy: 0.9918 - val_loss: 0.0364 - val_categorical_accuracy: 0.9895
Epoch 6/10
1500/1500 [==============================] - 49s 33ms/step - loss: 0.0198 - categorical_accuracy: 0.9936 - val_loss: 0.0474 - val_categoric

In [ ]:
best_model = tf.keras.models.load_model('/content/best_cnn_model.h5')

In [ ]:
best_model.evaluate(test_data, test_labels)

313/313 [==============================] - 2s 6ms/step - loss: 0.0307 - categorical_accuracy: 0.9903


[0.030704880133271217, 0.9902999997138977]